In [ ]:
import pandas as pd
import geopandas as gpd
import gtfs_functions
import yaml

In [ ]:
import matplotlib as mpl
mpl.rcParams["figure.dpi"] = 300

In [ ]:
hennepin_df = gpd.read_file("hennepin-county-property-map-cleaned.geojson")

In [ ]:
ramsey_df = gpd.read_file("ramsey-county-property-map-cleaned.geojson")

In [ ]:
anoka_df = gpd.read_file("anoka-county-property-map-cleaned.geojson")

In [ ]:
feed = gtfs_functions.Feed("gtfs.zip")

In [ ]:
lrt_route_numbers = ["901", "902"]
# brt_route_numbers = ["903", "904", "921", "923", "924"]
brt_route_numbers = ["904", "921", "923", "924"]
# commuter_rail_route_numbers = ["888"]
bus_routes = ["21", "63"]
route_numbers = lrt_route_numbers + brt_route_numbers + bus_routes

Note that the high frequency network is defined as "Parts of routes 2, 3, 6, 10, 11, 18, 54, 64, and all of routes 21 and 63, the METRO A Line, C Line, D Line, Blue Line, Green Line and Orange Line." Descriptively, the network has service at least every 15 minutes on weekdays between 6 AM and 7 PM, and on Saturdays between 9 AM and 6 PM. Thus, we want to find only the stops on these bus routes that are part of the high frequency network, and not all stops on other variants of these routes.

In [ ]:
stop_codes = set(feed.stop_times[feed.stop_times.route_id.isin(route_numbers)].stop_code.unique())
with open("high_frequency_stops.yaml") as f:
    stop_codes.update(yaml.safe_load(f))
stops = feed.stops[feed.stops.stop_code.isin(stop_codes)]
stops.explore()

In [ ]:
WKT = """PROJCS["NAD_1983_HARN_Adj_MN_Hennepin_Feet",
    GEOGCS["GCS_NAD_1983_HARN_Adj_MN_Hennepin",
        DATUM["D_NAD_1983_HARN_Adj_MN_Hennepin",
            SPHEROID["S_GRS_1980_Adj_MN_Hennepin",6378418.941,298.257222100883,
                AUTHORITY["ESRI","107726"]],
            AUTHORITY["ESRI","106726"]],
        PRIMEM["Greenwich",0,
            AUTHORITY["EPSG","8901"]],
        UNIT["degree",0.0174532925199433,
            AUTHORITY["EPSG","9122"]],
        AUTHORITY["ESRI","104726"]],
    PROJECTION["Lambert_Conformal_Conic_2SP"],
    PARAMETER["latitude_of_origin",44.7911111111111],
    PARAMETER["central_meridian",-93.3833333333333],
    PARAMETER["standard_parallel_1",44.8833333333333],
    PARAMETER["standard_parallel_2",45.1333333333333],
    PARAMETER["false_easting",500000],
    PARAMETER["false_northing",100000],
    UNIT["US survey foot",0.304800609601219,
        AUTHORITY["EPSG","9003"]],
    AXIS["Easting",EAST],
    AXIS["Northing",NORTH],
    AUTHORITY["ESRI","103734"]]
"""

In [ ]:
hennepin_projected = hennepin_df.to_crs(crs=WKT)
ramsey_projected = ramsey_df.to_crs(crs=WKT)
anoka_projected = anoka_df.to_crs(crs=WKT)
property_df = pd.concat([
    gpd.GeoDataFrame(geometry=hennepin_projected.geometry),
    gpd.GeoDataFrame(geometry=ramsey_projected.geometry),
    gpd.GeoDataFrame(geometry=anoka_projected.geometry),
])

In [ ]:
stops_projected = stops.to_crs(crs=WKT)

In [ ]:
buffered_stops = stops_projected.buffer(5280 / 2).unary_union
buffered_stops

In [ ]:
property_df["near_stop"] = property_df.geometry.make_valid().intersects(buffered_stops)

In [ ]:
property_df.near_stop.count(), property_df.near_stop.sum()

In [ ]:
ax = property_df.plot("near_stop", categorical=True)
ax.set_axis_off()